# NLP - HW7
### Miguel Bonilla

In [82]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get
import re
import contractions
from sklearn.feature_extraction.text import CountVectorizer

Cluster the reviews that you collected in homework 5, by doing the following:  
1. In Python, select any one of the clustering methods covered in this course. Run it over the
collection of reviews, and show at least two different ways of clustering the reviews, e.g.,
changing k in k-Means clustering or changing where you “cut” in Agnes or Diana.  
2. Try to write a short phrase to characterize (give a natural interpretation of) what each
cluster is generally centered on semantically. Is this hard to do in some cases? If so, make
note of that fact.  
3. Explain which of the two clustering results from question 1 is preferable (if one of them is),
and why.  
Submit all of your inputs and outputs and your code for this assignment, along with a brief written
explanation of your findings

In [2]:
### assign headers since IMDB rejects the requests without it
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.50'}

In [3]:
## load static URL list (from HW5)

url_list = pd.read_csv("../DS7337/mb_hw5_urls.csv")

In [4]:
# function goes through the table with the URLs to get each direct URL
# Parses through the content of each URL to grab the main review
# tokenizes the sentences of each review
# returns a dataframe with the movie title, review id, and the setence tokens
def grab_review(links_table):
    text = []
    for i in range(len(links_table)):
        review = get(links_table.url[i],headers)
        review_soup = BeautifulSoup(review.content, 'html.parser')
        text.append(review_soup.find(class_='text show-more__control').text)
    return(pd.DataFrame({'movie':links_table.movie,
                         'review':links_table.review,
                         'text':text                         
                        }))

In [5]:
review_text = grab_review(url_list)

In [101]:
special = ['\x96',':',',','-','(',')','[',']','–','/','#','``',';','.','&','"',"''",'?','!','....','--','...','*','..',"'"]
stop_words = nltk.corpus.stopwords.words('english')
special = stop_words + ["'s","'t","'d","'ll","'m","'re","'ve","n't"] +special

In [102]:
def normalize_list(review_list):
    term = [word_tokenize(term.lower()) for term in review_list]
    blank_list = []
    for i in range(len(review_list)):
        blank_list.append(' '.join([w for w in term[i] if w not in special]))
    return(np.array(blank_list))

In [103]:
norm_corpus = normalize_list(review_text.text)

In [104]:
cv = CountVectorizer(ngram_range=(1,2), min_df=10, max_df=0.8, stop_words=special)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix.shape

(100, 299)

In [113]:
from sklearn.cluster import KMeans
NUM_Clusters = 5
km = KMeans(n_clusters=NUM_Clusters, max_iter=1000,n_init=500,random_state=326).fit(cv_matrix)
km

KMeans(max_iter=1000, n_clusters=5, n_init=500, random_state=326)

In [114]:
from collections import Counter
Counter(km.labels_)

Counter({1: 7, 2: 57, 0: 28, 4: 4, 3: 4})